# Classificação Redes Neurais - Wisconsin Diagnostic Breast Cancer

## Descrição da atividade

- Nesta lista vamos fazer a predição para saber se um tumor de mama é benigno ou maligno a partir do dataset "Wisconsin Diagnostic Breast Cancer (WDBC)".
- Será utilizado 70% dos dados para treino e outros 30% para teste.
- Vamos criar modelos de aprendizado de máquina usando os seguintes algoritmos:
  - Perceptron
  - Adaline com Gradiente Descendente
  - Adaline com Gradiente Descentente Estocástico
  - Adaline com Gradiente Descendente Estocástico usando mini-batches de 20 elementos (mesmo do item anterior, mas agora com mini-batch).
- Os resultados obtidos entre os algoritmos acima usando a métrica acurácia serão comparados no final desta atividade.

## Importações

In [32]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron, SGDClassifier

## Carregamento e divisão dos dados

In [33]:
data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

## Normalização dos dados

In [34]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

## Perceptron

In [35]:
perceptron = Perceptron(random_state=5)
perceptron.fit(X_train_std, y_train)
y_pred_perceptron = perceptron.predict(X_test_std)
accuracy_perceptron = accuracy_score(y_test, y_pred_perceptron)

## Adaline com Gradiente Descendente

In [36]:
class Adaline(object):
    def __init__(self, eta=0.001, epoch=100):
        self.eta = eta
        self.epoch = epoch

    def fit(self, X, y):
        np.random.seed(16)
        self.weight_ = np.random.uniform(-1, 1, X.shape[1] + 1)
        self.error_ = []

        cost = 0
        for _ in range(self.epoch):
            output = self.activation_function(X)
            error = y - output

            self.weight_[0] += self.eta * sum(error)
            self.weight_[1:] += self.eta * X.T.dot(error)

            cost = 1./2 * sum((error**2))
            self.error_.append(cost)

        return self

    def net_input(self, X):
        """Calculo da entrada z"""
        return np.dot(X, self.weight_[1:]) + self.weight_[0]

    def activation_function(self, X):
        """Calculo da saida da funcao g(z)"""
        return self.net_input(X)

    def predict(self, X):
        """Retornar valores binaros 0 ou 1"""
        return np.where(self.activation_function(X) >= 0.0, 1, -1)

adaline = Adaline()
adaline.fit(X_train_std, y_train)
y_pred_adaline = adaline.predict(X_test_std)
accuracy_adaline = accuracy_score(y_test, y_pred_adaline)


## Adaline com Gradiente Descendente Estocástico

In [37]:
sgd = SGDClassifier(random_state=7)
sgd.fit(X_train_std, y_train)
y_pred_sgd = sgd.predict(X_test_std)
accuracy_sgd = accuracy_score(y_test, y_pred_sgd)


## Adaline com Gradiente Descendente Estocástico (mini-batches de 20 elementos)

In [38]:
from sklearn.linear_model import SGDClassifier

sgd_mini_batch = SGDClassifier(random_state=7, max_iter=1000, tol=1e-3)
sgd_mini_batch.fit(X_train_std, y_train)

batch_size = 20
n_samples = X_train_std.shape[0]

n_batches = n_samples // batch_size
if n_samples % batch_size != 0:
    n_batches += 1

accuracy_sgd_mini_batch = 0.0

for i in range(n_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    X_batch = X_train_std[start_idx:end_idx]
    y_batch = y_train[start_idx:end_idx]
    y_pred_batch = sgd_mini_batch.predict(X_batch)
    accuracy_sgd_mini_batch += accuracy_score(y_batch, y_pred_batch) * len(X_batch)

accuracy_sgd_mini_batch /= n_samples


## Impressão dos resultados

In [39]:
print("Perceptron: {:.2f}%".format(accuracy_perceptron * 100))
print("Adaline com Gradiente Descendente: {:.2f}%".format(accuracy_adaline * 100))
print("Adaline com Gradiente Descendente Estocástico: {:.2f}%".format(accuracy_sgd * 100))
print("Adaline com Gradiente Descendente Estocástico (mini-batches de 20 elementos): {:.2f}%".format(accuracy_sgd_mini_batch * 100))


Perceptron: 98.25%
Adaline com Gradiente Descendente: 59.06%
Adaline com Gradiente Descendente Estocástico: 97.08%
Adaline com Gradiente Descendente Estocástico (mini-batches de 20 elementos): 98.49%
